## ETL Run Notebook  - Run scripts & Test w/ sql queries

### Run create_table.py to see if tables can be created property

- Make sure you open a RedShift cluster and make it publicly accessible

In [8]:
 pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2 

try:
    %run create_table.py 
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS co2emssion_stage(
        country varchar,
        code varchar(3),
        year int, 
        CO2emission BIGINT
                );
        
CREATE TABLE IF NOT EXISTS population_stage(
        'country name' varchar,
        year int, 
        population int
        );
        


SyntaxError: syntax error at or near "'country name'"
LINE 2:         'country name' varchar,
                ^


In [2]:
try:
    %run etl.py 
except Exception as e:
    print(e)

UndefinedColumn: column "dt" of relation "surfacetemp_stage" does not exist


#### Test tables with some Queries